In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(output_distribution='uniform')

In [2]:
DATASET_DIRECTORY = '/kaggle/input/unb-cic-iot-dataset/wataiData/csv/CICIoT2023/'

In [3]:
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets)*.67)]
test_sets = df_sets[int(len(df_sets)*.67):]

In [4]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count', 
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight', 
]
y_column = 'label'

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
def build_CNN_model(input_shape):
    model = Sequential([
        Conv1D(32, 3, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(34, activation='softmax')  # Output layer with softmax activation for multiclass
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

2024-03-17 06:49:34.299625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 06:49:34.299721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 06:49:34.436868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
DL_models = [
        build_CNN_model(input_shape=(len(X_columns),1))
            ]


In [7]:
DL_names = [
        "CNN",
]

In [8]:

for train_set in tqdm(training_sets):
        d = pd.read_csv(DATASET_DIRECTORY + train_set)
        d[X_columns] = scaler.fit_transform(d[X_columns])
        X_train = d[X_columns].values
        y_train = d[y_column].values
        from sklearn.preprocessing import LabelEncoder

        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        for model in (DL_models):
            model.fit(X_train, y_train_encoded, epochs=3, batch_size=40, validation_split=0.3)
        del d

  0%|          | 0/113 [00:00<?, ?it/s]

Epoch 1/3
  84/4177 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.2818 - loss: 2.8118

I0000 00:00:1710658193.559860      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710658193.578733      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4173/4177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8258 - loss: 0.5724

W0000 00:00:1710658202.178305      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


...

 98%|█████████▊| 111/113 [1:17:32<01:14, 37.37s/it]

Epoch 1/3
4712/4712 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9839 - loss: 0.0553 - val_accuracy: 0.9867 - val_loss: 0.0502
Epoch 2/3
4712/4712 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9859 - loss: 0.0482 - val_accuracy: 0.9855 - val_loss: 0.0474
Epoch 3/3
4712/4712 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9850 - loss: 0.0542 - val_accuracy: 0.9878 - val_loss: 0.0429


 99%|█████████▉| 112/113 [1:18:21<00:40, 40.72s/it]

Epoch 1/3
4438/4438 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9836 - loss: 0.0626 - val_accuracy: 0.9866 - val_loss: 0.0430
Epoch 2/3
4438/4438 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9846 - loss: 0.0514 - val_accuracy: 0.9879 - val_loss: 0.0405
Epoch 3/3
4438/4438 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9849 - loss: 0.0523 - val_accuracy: 0.9883 - val_loss: 0.0416


100%|██████████| 113/113 [1:19:00<00:00, 41.95s/it]


In [9]:
original_labels=['DDoS-RSTFINFlood','DDoS-PSHACK_Flood','DDoS-SYN_Flood','DDoS-UDP_Flood','DDoS-TCP_Flood','DDoS-ICMP_Flood',
'DDoS-SynonymousIP_Flood','DDoS-ACK_Fragmentation','DDoS-UDP_Fragmentation','DDoS-ICMP_Fragmentation',
'DDoS-SlowLoris','DDoS-HTTP_Flood','DoS-UDP_Flood','DoS-SYN_Flood','DoS-TCP_Flood','DoS-HTTP_Flood','Mirai-greeth_flood','Mirai-greip_flood','Mirai-udpplain',
'Recon-PingSweep','Recon-OSScan','Recon-PortScan','VulnerabilityScan',
'Recon-HostDiscovery',
'DNS_Spoofing',
'MITM-ArpSpoofing',
'BenignTraffic',
'BrowserHijacking',
'Backdoor_Malware',
'XSS',
'Uploading_Attack',
'SqlInjection',
'CommandInjection',
'DictionaryBruteForce']

In [10]:
import numpy as np
y_test = []
d_preds = {i:[] for i in range(len(DL_models))}
for test_set in tqdm(test_sets):
    d_test = pd.read_csv(DATASET_DIRECTORY + test_set)
    d_test[X_columns] = scaler.fit_transform(d_test[X_columns])
    y_test += list(d_test[y_column].values)
    for i in range(len(DL_models)):
        model = DL_models[i]
        y_preds = list(model.predict(d_test[X_columns]))
        d_preds[i] = d_preds[i] + y_preds
    del y_preds
    del d_test
    

  0%|          | 0/56 [00:00<?, ?it/s]

14110/14110 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step


  2%|▏         | 1/56 [00:33<30:55, 33.73s/it]

7497/7497 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step


  4%|▎         | 2/56 [00:51<21:53, 24.32s/it]

...

 96%|█████████▋| 54/56 [20:05<01:00, 30.27s/it]

7116/7116 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step


W0000 00:00:1710664166.338517      75 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
 98%|█████████▊| 55/56 [20:23<00:26, 26.51s/it]

7336/7336 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step


100%|██████████| 56/56 [20:42<00:00, 22.18s/it]


In [11]:
len(d_preds)

1

In [12]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

for k, v in d_preds.items():
    if isinstance(v, list):
        v = np.array(v)
    y_preds = np.argmax(v, axis=1)
    label_encoder.fit(original_labels)
    y_preds = label_encoder.inverse_transform(y_preds)
    from sklearn.metrics import classification_report
    
    print(f"{DL_names[k]}")
    print(classification_report(y_test,y_preds))
    print('accuracy_score: ', accuracy_score(y_preds, y_test))
    print('recall_score: ', recall_score(y_preds, y_test, average='macro'))
    print('precision_score: ', precision_score(y_preds, y_test, average='macro'))
    print('f1_score: ', f1_score(y_preds, y_test, average='macro'))


CNN
                         precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.00      0.00      1131
          BenignTraffic       0.78      0.96      0.86    387031
       BrowserHijacking       0.00      0.00      0.00      2041
       CommandInjection       0.00      0.00      0.00      1863
 DDoS-ACK_Fragmentation       0.99      0.99      0.99    100231
        DDoS-HTTP_Flood       0.95      0.89      0.92     10131
        DDoS-ICMP_Flood       1.00      1.00      1.00   2537902
DDoS-ICMP_Fragmentation       0.95      0.98      0.96    160098
      DDoS-PSHACK_Flood       1.00      1.00      1.00   1445379
       DDoS-RSTFINFlood       1.00      1.00      1.00   1426053
         DDoS-SYN_Flood       1.00      1.00      1.00   1432485
         DDoS-SlowLoris       0.80      0.95      0.87      8323
DDoS-SynonymousIP_Flood       1.00      1.00      1.00   1267869
         DDoS-TCP_Flood       1.00      1.00      1.00   1586183
         DDoS-UDP_Fl

In [13]:
!rm -rf /kaggle/working/*

In [14]:
from keras.models import model_from_json
import os

# Save model
def save_model(model, model_name):
    # Serialize model to JSON
    model_json = model.to_json()
    with open(model_name + ".json", "w") as json_file:
        json_file.write(model_json)

    # Serialize weights to HDF5
    model.save_weights(model_name + ".weights.h5")
modelCNN = build_CNN_model(input_shape=(len(X_columns), 1))
save_model(modelCNN, "/kaggle/working/CNN34classfinal")